<a href="https://colab.research.google.com/github/Kirutkar/airesidency/blob/main/rag_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install llama_index

In [3]:
import openai
from google.colab import userdata

# Retrieve the OpenAI API key from Google Colab secrets
openai.api_key = userdata.get('openai')

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()
response = query_engine.query("who is mentioned about in document")
print(response)

Paul Graham, Maria Daniels, Jessica Livingston, Robert, Trevor, Julian


In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()
response = query_engine.query("how much covered for dental")
print(response)

Emergency Dental Treatment is covered for amounts ranging from USD 200 to USD 1000 in multiples of 100.


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7983ef2916a9d61b74.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [23]:
# 🧩 Install required packages
!pip install llama-index gradio PyMuPDF

# 📦 Imports
import os
import gradio as gr
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from pathlib import Path

# 📁 Ensure data folder exists
Path("data").mkdir(exist_ok=True)

# 📖 Load uploaded documents
def build_index():
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    return index.as_query_engine()

# 🧠 Check if any documents are already present
if any(Path("data").iterdir()):
    query_engine = build_index()
else:
    class DummyQueryEngine:
        def query(self, query_str):
            return "📂 Please upload a document to get started."
    query_engine = DummyQueryEngine()

# 🔍 Query documents
def ask_index(query):
    if not query:
        return "❗ Please enter a question."
    if isinstance(query_engine, DummyQueryEngine):
        return query_engine.query(query)
    return str(query_engine.query(query))

# 📤 Handle upload
def upload_and_refresh(file):
    if file is None:
        return "⚠️ No file uploaded."
    file_path = os.path.join("data", file.name)
    file.save(file_path)
    global query_engine
    query_engine = build_index()
    return f"✅ Uploaded & Indexed: {file.name}"

# 📌 Summarize document
def summarize_doc():
    if isinstance(query_engine, DummyQueryEngine):
        return query_engine.query("Summarize the uploaded documents.")
    return str(query_engine.query("Summarize the uploaded documents."))

# 🚀 Launch Gradio App
def launch():
    with gr.Blocks(title="RAG App with LlamaIndex", css="""
        body { background-color: #f0f4f8; }
        .title-row { display: flex; align-items: center; gap: 18px; padding-left: 10px; }
        .section { background: white; padding: 18px; border-radius: 12px; box-shadow: 0 0 12px rgba(0,0,0,0.06); margin-bottom: 20px; }
        h1, h2, h3 { color: #002855; }
    """) as app:

        # ✅ HEADER SECTION
        with gr.Row():
            with gr.Column(scale=0, min_width=100):
                gr.Image("data/llama_logo_ui.png", width=70, height=70, show_label=False)
            with gr.Column(scale=6):
                gr.Markdown("""
                    <div class='title-row'>
                        <h1 style='font-size: 2.2em; margin: 0;'>RAG Application with LlamaIndex</h1>
                    </div>
                    <div style='margin-top: -8px; font-size: 1.05em;'>📄 Ask questions from documents using Retrieval-Augmented Generation (RAG)</div>
                """)

        # 📘 TAB 1 — Insurance
        with gr.Tab("📘 Insurance Summary"):
            with gr.Column(elem_classes="section"):
                gr.Markdown("### 🛡️ General Insurance Coverage")
                gr.Markdown("This section helps you ask questions or summarize the contents of the insurance policy documents.")
                with gr.Row():
                    question = gr.Textbox(label="Ask your question", placeholder="e.g., Summarize the insurance details in the document")
                    ask_btn = gr.Button("Submit")
                answer = gr.Textbox(label="Response", lines=6)
                ask_btn.click(fn=ask_index, inputs=question, outputs=answer)

        # 🧠 TAB 2 — Paul Graham
        with gr.Tab("🧠 Paul Graham"):
            with gr.Column(elem_classes="section"):
                gr.Markdown("### 🧠 Paul Graham Essays")
                gr.Markdown("Ask questions based on essays or content by Paul Graham.")
                with gr.Row():
                    paul_q = gr.Textbox(label="Ask about Paul Graham", placeholder="e.g., Summarize Paul Graham's main message")
                    ask_btn2 = gr.Button("Ask")
                paul_ans = gr.Textbox(label="Response", lines=6)
                ask_btn2.click(fn=ask_index, inputs=paul_q, outputs=paul_ans)

                gr.Markdown("#### 💬 Example Prompts:")
                gr.Examples(
                    examples=["Summarize Paul Graham's ideas", "What does Paul say about startup ideas?"],
                    inputs=paul_q,
                )

        # 📤 TAB 3 — Upload & Ask
        with gr.Tab("📤 Upload & Ask Your File"):
            with gr.Column(elem_classes="section"):
                gr.Markdown("### 📤 Upload a Document")
                gr.Markdown("Upload a `.pdf` or `.txt` file. Then ask questions or request a summary.")
                file_input = gr.File(label="Upload PDF or TXT")
                status = gr.Textbox(label="Status", interactive=False)
                file_input.change(fn=upload_and_refresh, inputs=file_input, outputs=status)

                gr.Markdown("#### 💬 Ask a Question")
                user_q = gr.Textbox(label="Your Query")
                user_btn = gr.Button("Ask")
                user_ans = gr.Textbox(label="Response", lines=6)
                user_btn.click(fn=ask_index, inputs=user_q, outputs=user_ans)

                gr.Markdown("#### 📌 Summarize the File")
                summarize = gr.Button("Summarize Uploaded Document")
                summary_output = gr.Textbox(label="Summary", lines=6)
                summarize.click(fn=summarize_doc, outputs=summary_output)

    app.launch()

# 🚀 Run the app
launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dcda6abea9a928432a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
